In [ ]:
!pip install tensorflow

In [13]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Layer, Embedding, MultiHeadAttention, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [14]:
class SimpleTransformer(Model):
    def __init__(self, vocab_size, d_model, nhead, num_layers):
        super(SimpleTransformer, self).__init__()
        self.embedding = Embedding(vocab_size, d_model)
        self.transformer = [
            MultiHeadAttention(num_heads=nhead, key_dim=d_model)
            for _ in range(num_layers)
        ]
        self.fc = Dense(vocab_size)

    def call(self, inputs, training):
        src, tgt = inputs
        src = self.embedding(src)
        tgt = self.embedding(tgt)
        for layer in self.transformer:
            tgt = layer(tgt, src, src, training=training)
        x = self.fc(tgt)
        return x


In [15]:
input_texts = ["this is a simple example", "this is another example"]
target_texts = ["simple example", "another example"]

tokenizer = Tokenizer(filters="", lower=True, char_level=False)
tokenizer.fit_on_texts(input_texts + target_texts)

input_seqs = tokenizer.texts_to_sequences(input_texts)
target_seqs = tokenizer.texts_to_sequences(target_texts)

input_seqs = pad_sequences(input_seqs, padding="post")
target_seqs = pad_sequences(target_seqs, padding="post")

vocab_size = len(tokenizer.word_index) + 1

In [16]:
d_model = 64
nhead = 4
num_layers = 2

model = SimpleTransformer(vocab_size, d_model, nhead, num_layers)

In [17]:
epochs = 100
batch_size = 2
loss_fn = SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer=Adam(), loss=loss_fn)
model.fit(x=[input_seqs, target_seqs[:, :-1]], y=target_seqs[:, 1:], batch_size=batch_size, epochs=epochs)

Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 1.9455
Epoch 2/100
1/1 [==============================] - 0s 19ms/step - loss: 1.9070
Epoch 3/100
1/1 [==============================] - 0s 16ms/step - loss: 1.8663
Epoch 4/100
1/1 [==============================] - 0s 17ms/step - loss: 1.8203
Epoch 5/100
1/1 [==============================] - 0s 17ms/step - loss: 1.7666
Epoch 6/100
1/1 [==============================] - 0s 17ms/step - loss: 1.7040
Epoch 7/100
1/1 [==============================] - 0s 18ms/step - loss: 1.6313
Epoch 8/100
1/1 [==============================] - 0s 19ms/step - loss: 1.5478
Epoch 9/100
1/1 [==============================] - 0s 16ms/step - loss: 1.4529
Epoch 10/100
1/1 [==============================] - 0s 15ms/step - loss: 1.3463
Epoch 11/100
1/1 [==============================] - 0s 22ms/step - loss: 1.2283
Epoch 12/100
1/1 [==============================] - 0s 16ms/step - loss: 1.0996
Epoch 13/100
1/1 [==============================] -

In [18]:
def summarize(input_text, model, tokenizer, max_summary_length=10):
    input_seq = tokenizer.texts_to_sequences([input_text])
    input_seq = pad_sequences(input_seq, padding="post")
    summary_seq = np.zeros((1, max_summary_length), dtype=np.int32)

    for i in range(max_summary_length):
        logits = model.predict([input_seq, summary_seq])
        predicted_token = np.argmax(logits[0, i])
        summary_seq[0, i] = predicted_token
        if predicted_token == 0:
            break

    summary_text = tokenizer.sequences_to_texts(summary_seq)
    return summary_text[0]

In [19]:
input_text = "this is a simple example"
summary = summarize(input_text, model, tokenizer)
print(f"Input text: {input_text}")
print(f"Generated summary: {summary}")

1/1 [==============================] - 0s 23ms/step
Input text: this is a simple example
Generated summary: example example example example example example example example example example
